In [1]:
from os import getenv

In [18]:
import pyodbc
import pandas as pd

In [9]:
def database_connection():
    conn = pyodbc.connect(
        r'DRIVER={SQL Server Native Client 11.0};'
        r'SERVER=localhost;'
        r'DATABASE=seis603_pricing_project;'
        r'UID=python_dev;'
        r'PWD=python_dev;'
        )
    cursor = conn.cursor()
    return cursor
db_connection = database_connection()

In [42]:
sql = """\
        SELECT uit.i_id
		,i.i_name
		,s.s_id
		,s.s_name
		,MAX(iph.i_price) AS Current_Price
		--,MAX(iph.i_date) AS Lastest_Date
FROM dbo.UserItemTracking AS uit
INNER JOIN dbo.ItemPriceHistory AS iph ON uit.i_id = iph.i_id
INNER JOIN dbo.Store AS s ON iph.s_id = s.s_id
INNER JOIN dbo.Item AS i ON uit.i_id = i.i_id
WHERE uit.u_id=100
GROUP BY uit.i_id
	,i.i_name
       , s.s_id
		,s.s_name
	  
	   ;
       """

In [43]:
db_connection.execute(sql) #executing sproc
price_lists = db_connection.fetchall()#[0] #fetchone will only return first result

In [44]:
labels = ['item id','item name','store id','store name','current price']

In [45]:
df_current_price = pd.DataFrame.from_records(price_lists, columns=labels) #create dataframe from list

In [47]:
pvt_current_prices =  df_current_price.pivot(index='store name',columns='item name',values = 'current price' )

In [48]:
print(pvt_current_prices)

item name      Adrenaline GTS 17 Radian R100 Car Seat
store name                                           
Brooks Running             78.00                 None
Target                      None               179.99
Toys r Us                   None               239.99


In [41]:
pivoted = pvt_current_prices.pivot('store name', 'item name')

KeyError: 'store name'